# 家計・生活・労働データの取得・可視化・分析


In [1]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import json
import requests
# データ処理に用いるライブラリ
import numpy as np
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
    create_hierarchy_dataframe
)

In [4]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")
bls_api_key = os.getenv("BLS_API_KEY")

In [6]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch08").resolve()

## 家計に関する統計データ

### 家計調査

- [家計調査](https://www.stat.go.jp/data/kakei/)

#### 家計収支の前年同月比と月次推移

- [家計調査　収支項目分類](https://www.stat.go.jp/data/kakei/9.html)


In [7]:
monthly_statsDataId = "0002070010"
monthly_meta = get_metainfo(appId, monthly_statsDataId)
monthly_metadata = monthly_meta["GET_META_INFO"]["METADATA_INF"]
monthly_total_num = monthly_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
monthly_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


3916136

In [21]:
monthly_metadata["CLASS_INF"]["CLASS_OBJ"]

[{'@id': 'tab',
  '@name': '表章項目',
  'CLASS': {'@code': '01', '@name': '金額', '@level': ''}},
 {'@id': 'cat01',
  '@name': '用途分類',
  '@description': '「事業・内職収入」は「事業・内職収入（農林漁業収入を除く）（～2019 年）」、「事業・内職収入（農林漁業収入を含む）（2020 年～）」に分割しています。',
  'CLASS': [{'@code': '001',
    '@name': '世帯数分布（抽出率調整）',
    '@level': '1',
    '@unit': '一万分比'},
   {'@code': '002', '@name': '集計世帯数', '@level': '1', '@unit': '世帯'},
   {'@code': '003', '@name': '世帯人員', '@level': '1', '@unit': '人'},
   {'@code': '004',
    '@name': '18歳未満人員',
    '@level': '2',
    '@unit': '人',
    '@parentCode': '003'},
   {'@code': '005',
    '@name': '65歳以上人員',
    '@level': '2',
    '@unit': '人',
    '@parentCode': '003'},
   {'@code': '006',
    '@name': '65歳以上無職者人員',
    '@level': '3',
    '@unit': '人',
    '@parentCode': '005'},
   {'@code': '007', '@name': '有業人員', '@level': '1', '@unit': '人'},
   {'@code': '008', '@name': '世帯主の配偶者のうち女の有業率', '@level': '1', '@unit': '％'},
   {'@code': '009', '@name': '世帯主の年齢', '@level': '1', '@unit': 

In [ ]:
# 分類名の一覧を確認
display(pd.DataFrame(monthly_metadata["CLASS_INF"]["CLASS_OBJ"])[["@id", "@name"]])

# 特定の分類（例えば 1番目の 'cat01'：用途分類など）の具体的な項目を確認
# 1. monthly_metadata["CLASS_INF"]["CLASS_OBJ"] は、全ての分類項目（世帯区分、用途分類など）のリストです。
# 2. [1] は、そのリストの2番目（インデックス1）の要素を取得しています。（通常、家計調査では「用途分類」がここに来ます）
# 3. ["CLASS"] は、そのカテゴリに含まれる具体的な分類項目（例：「食料」「住居」など）のデータを取り出しています。
# 4. pd.DataFrame(...) で、それらの項目を DataFrame 形式に変換して読みやすくしています。
pd.DataFrame(monthly_metadata["CLASS_INF"]["CLASS_OBJ"][1]["CLASS"])

,@id,@name
0,tab,表章項目
1,cat01,用途分類
2,cat02,世帯区分
3,cat03,世帯主の年齢階級
4,area,地域区分
5,time,時間軸（月次）


,@code,@name,@level,@unit,@parentCode
0,001,世帯数分布（抽出率調整）,1,一万分比,NaN
1,002,集計世帯数,1,世帯,NaN
2,003,世帯人員,1,人,NaN
3,004,18歳未満人員,2,人,003
4,005,65歳以上人員,2,人,003
...,...,...,...,...,...
260,261,一括払購入借入金純減率,3,％,251
261,262,財産純増率,3,％,251
262,263,エンゲル係数,1,％,NaN
263,264,年間収入,1,万円,NaN


In [22]:
meta_cat01 = pd.DataFrame(
    monthly_metadata["CLASS_INF"]["CLASS_OBJ"][1]["CLASS"]
)
meta_cat01.iloc[55:65, :]  # 支払のcodeがわかる行を抽出しています

,@code,@name,@level,@unit,@parentCode
55,053,一括払購入借入金,3,円,044
56,054,財産売却,3,円,044
57,055,実収入以外の受取のその他,3,円,044
58,056,繰入金,2,円,018
59,057,支払,1,円,NaN
60,058,実支出,2,円,057
61,059,消費支出,3,円,058
62,060,食料,4,円,059
63,061,穀類,5,円,060
64,062,米,6,円,061


In [23]:
monthly_params = {
    "lvCat01": "4",  # "用途分類"のlevelを"4"に絞る
    "cdCat02": "04",  # "世帯区分"を"二人以上の世帯のうち勤労者世帯（2000年～）"に絞る
    "cdCat03": "A00",  # "世帯主の年齢階級"を"平均"に絞る
    "cdTimeFrom": "2018000101",  # "時間軸（月次）"を2018年以降に絞る
}
monthly_data = get_statsdata(appId, monthly_statsDataId, params=monthly_params)
monthly_value = colname_to_japanese(cleansing_statsdata(monthly_data))
monthly_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '用途分類コード', '世帯区分コード', '世帯主の年齢階級コード', '地域区分コード', '時間軸（月次）コード',
       '単位', '値', '表章項目', '表章項目階層レベル', '用途分類', '用途分類階層レベル', '用途分類単位',
       '用途分類親コード', '世帯区分', '世帯区分階層レベル', '世帯主の年齢階級', '世帯主の年齢階級階層レベル', '地域区分',
       '地域区分階層レベル', '時間軸（月次）', '時間軸（月次）階層レベル'],
      dtype='object')

In [85]:
# monthly_value という元のデータに対して、新しい列（年、月）を追加した monthly_df を作成します
monthly_df = monthly_value.assign(**{
    # 「時間軸（月次）コード」（例：2024000101）を数値として扱い、
    # 1,000,000 で割ることで、先頭の4桁（例：2024）を「年」として取り出します
    "年": monthly_value["時間軸（月次）コード"].astype(int) // 1_000_000,
    
    # 「時間軸（月次）コード」を文字列として扱い、
    # 後ろから2文字（例：01）を切り取って数値に変換し、「月」として取り出します
    "月": monthly_value["時間軸（月次）コード"].str[-2:].astype(int)
    
# 最後に、解析に必要な4つの列（「年」「月」「用途分類」「値」）だけを抽出して並べ替えます
})[["年", "月", "用途分類", "値"]]
monthly_df

,年,月,用途分類,値
0,2018,1,勤め先収入,420845.0
1,2018,2,勤め先収入,419498.0
2,2018,3,勤め先収入,431497.0
3,2018,4,勤め先収入,430698.0
4,2018,5,勤め先収入,426479.0
...,...,...,...,...
3154,2025,7,有価証券純購入率,1.1
3155,2025,8,有価証券純購入率,0.8
3156,2025,9,有価証券純購入率,0.4
3157,2025,10,有価証券純購入率,0.9


##### 前年同月比の折れ線グラフ



In [25]:
# 2019年から2024年まで6年分の年範囲を作成
year = 2024
year_range = range(year - 5, year + 1)
# 使用する色をPlotlyの定義済みカラーパレットから選択
yoy_colors = px.colors.qualitative.Plotly[: len(year_range)]
# X軸の目盛りテキスト用に"1月"から"12月"までの文字列を用意
months = [f"{mm}月" for mm in range(1, 13)]
# 使用する用途分類のリストを用意
yoy_cats = ["食料", "光熱・水道", "教養娯楽", "勤め先収入"]
# 2*2 のサブプロット用の行・列のペア
row_col_pair = [(i, j) for i in range(1, 3) for j in range(1, 3)]
# 2*2のサブプロットを作成
yoy_fig = make_subplots(rows=2, cols=2, subplot_titles=yoy_cats)
# 各年度ごとに各用途分類のトレースを追加
for cat, (row, col) in zip(yoy_cats, row_col_pair):
    for n, yyyy in enumerate(year_range):
        # 指定した用途分類と年に対する条件を作成
        yoy_cond = monthly_df["用途分類"] == cat
        yoy_cond &= monthly_df["年"] == yyyy
        # 最初の用途分類のトレースのみに凡例を表示
        show_legend = True if (row == 1 and col == 1) else False
        # 折れ線グラフのトレースをサブプロットに追加
        yoy_fig.add_trace(
            go.Scatter(
                x=list(range(1, 13)),
                y=monthly_df.loc[yoy_cond, "値"],
                name=f"{yyyy}年",
                marker={"color": yoy_colors[n]},
                showlegend=show_legend,  # 最初のトレースのみ凡例を表示
            ),
            row=row, col=col,
        )
        # X軸とY軸の設定を更新
        yoy_fig.update_xaxes(
            tickvals=list(range(1, 13)),  # X軸の目盛り値。1月から12月までの数値
            ticktext=months,  # X軸の目盛りテキスト。"1月"から"12月"までの文字列
            row=row, col=col,
        )
        yoy_fig.update_yaxes(
            tickformat=",.0f",
            ticksuffix="円",
            row=row, col=col,
        )
yoy_fig.update_layout(width=1000, height=800,
    legend={
        "orientation": "h",  # "h" は水平(horizontal)を意味し、凡例の項目が水平方向に並ぶ
        "x": 0.5,  # 0.5 はグラフの水平方向の中央
        "y": -0.1,  # -0.1 は、グラフ領域外の下部(グラフの下端からさらに10%下)に凡例を配置
        "xanchor": "center",  # 凡例の中央がxで指定された位置に配置
        "yanchor": "top",  # 凡例の上端がyで指定された位置に配置
    },
)
yoy_fig.show()

##### 月次推移とCOVID-19



In [26]:
covid_path = (current_dir / "data" / "ch07").resolve() / "severe_cases_daily.csv"
#covid_path = "https://covid19.mhlw.go.jp/public/opendata/severe_cases_daily.csv"
daily_covid = pd.read_csv(
    covid_path,
    parse_dates=["Date"],
)
monthly_covid = (daily_covid[["Date", "ALL"]]
    .rename(columns={"ALL": "重症者数の推移"})
    .set_index("Date")
    .resample("MS").sum()
    .reset_index()
)

In [86]:
daily_covid

,Date,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,...,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa
0,2020-05-09,248,28.0,0.0,0,0,0,0,0,3,...,2,0,0,1,0,2,0,0,0,NaN
1,2020-05-10,222,26.0,0.0,0,0,0,0,0,3,...,0,0,0,1,0,2,0,0,0,NaN
2,2020-05-11,248,24.0,0.0,0,0,0,0,0,2,...,1,0,0,1,0,2,0,0,0,1.0
3,2020-05-12,254,19.0,0.0,0,0,0,2,0,2,...,0,0,11,1,0,2,0,0,0,1.0
4,2020-05-13,241,18.0,0.0,0,0,0,2,2,2,...,0,0,11,0,0,1,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2023-05-05,78,4.0,1.0,1,1,0,0,0,2,...,1,0,2,0,0,1,0,0,1,4.0
1092,2023-05-06,79,4.0,1.0,1,1,0,0,0,2,...,1,0,2,0,0,1,0,0,1,4.0
1093,2023-05-07,82,4.0,2.0,1,3,0,0,0,2,...,1,0,1,0,0,1,0,0,1,5.0
1094,2023-05-08,71,4.0,1.0,1,0,0,0,0,2,...,1,0,1,0,0,1,0,0,1,5.0


In [88]:
# このセルは家計調査のデータ（monthly_df）とCOVID-19のデータ（mothly_covid）が詠み込まれていないと動きません

# "年"列と"月"列から"年月"の日付型データを作成して新しい列に追加
monthly_df = monthly_df.assign(**{
    "年月": pd.to_datetime(
        monthly_df["年"].astype(str) + "-" + monthly_df["月"].astype(str) + "-01"
    )
})
# 可視化する用途分類をリストで指定
cats = ["光熱・水道", "教養娯楽"]
colors = px.colors.qualitative.Plotly
# 1つのプロットに複数のy軸を持つサブプロットを作成
monthly_fig = make_subplots(specs=[[{"secondary_y": True}]])
# 用途分類ごとに処理を実行
for n, cat in enumerate(cats):
    # 指定した用途分類のデータを選択し、"年月"でソート
    tmp_df = monthly_df[monthly_df["用途分類"] == cat].sort_values("年月")
    # 選択した用途分類の平均値を計算
    monthly_mean = tmp_df["値"].mean()
    # 折れ線グラフを追加
    monthly_fig.add_trace(
        go.Scatter(
            x=tmp_df["年月"],
            y=tmp_df["値"],
            name=cat,
            mode="lines+markers",
            marker={"color": colors[n]},
        ),
        secondary_y=False,  # このトレースは第一のY軸を使用
    )
    # 平均値のラインを追加
    monthly_fig.add_trace(
        go.Scatter(
            x=[monthly_df["年月"].min(), monthly_df["年月"].max()],  # 全期間をカバー
            y=[monthly_mean, monthly_mean],  # 平均値で固定
            name=cat + "(期間平均)",
            mode="lines",
            marker={"color": colors[n]},
            opacity=0.5,  # 半透明で表示
        ),
        secondary_y=False,  # このトレースも第一のY軸を使用
    )
# COVID-19重症者数の棒グラフを追加
monthly_fig.add_trace(
    go.Bar(
        x=monthly_covid["Date"],
        y=monthly_covid["重症者数の推移"],
        name="COVID-19重症者数",
        marker={"color": colors[len(cats)]},  # 新しい色を指定
        opacity=0.5,  # 半透明で表示
    ),
    secondary_y=True,  # このトレースは第二のY軸を使用
)
monthly_fig.update_layout(width=1200, height=450, font={"size": 16},
    xaxis={"ticksuffix": "年"},
    yaxis={"tickformat": ",.0f", "ticksuffix": "円"},
)
monthly_fig.update_yaxes(secondary_y=True, tickformat=",.0f", ticksuffix="人")
monthly_fig.update_layout(
    title_text="COVID-19重傷者数と家計支出の月次推移",
    title_x=0.5
)
monthly_fig.show()

#### 年齢ごと収支



In [28]:
annual_statsDataId = "0002070011"
annual_meta = get_metainfo(appId, annual_statsDataId)
annual_metadata = annual_meta["GET_META_INFO"]["METADATA_INF"]
annual_total_num = annual_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
annual_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


322513

In [91]:
yyyy = 2024
age_params = {
    "cdCat01": "009",  # 用途分類を世帯主の年齢に絞る
    "cdCat02": "04",  # 世帯区分を二人以上の世帯のうち勤労者世帯（2000年～）に絞る
    "cdTime": f"{yyyy}000000",
}
age_data = get_statsdata(appId, annual_statsDataId, params=age_params)
age_value = colname_to_japanese(cleansing_statsdata(age_data))
age_cond = ~age_value["世帯主の年齢階級"].isin(["平均", "65歳以上"])
cols = ["時間軸（年次）", "世帯主の年齢階級", "値"]
age_df = age_value.loc[age_cond, cols].rename(columns={"値": "世帯主の年齢"})
age_df.head()

e-Stat APIエラー (Status 0): 正常に終了しました。


,時間軸（年次）,世帯主の年齢階級,世帯主の年齢
1,2024年,34歳以下,31.0
2,2024年,35～39歳,37.1
3,2024年,40～44歳,42.0
4,2024年,45～49歳,47.1
5,2024年,50～54歳,51.9


In [30]:
#  世帯主の年齢階級ごとの家計項目のデータを取得するための関数を用意
def get_filtered_statsdata(
        app_id: str, stats_id: str, cat01: str, value_name: str
    ) -> pd.DataFrame:
    params = {
        "cdCat01": cat01,
        "cdCat02": "04",
        "cdTime": f"{yyyy}000000",
    }
    data = get_statsdata(app_id, stats_id, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    cols = ["時間軸（年次）", "世帯主の年齢階級", "値"]
    df = value.loc[~value["世帯主の年齢階級"].isin(["平均", "65歳以上"]), cols]
    return df.rename(columns={"値": value_name})

# 各家計項目のデータ取得
income_df = get_filtered_statsdata(appId, annual_statsDataId, "233", "可処分所得")
expenditure_df = get_filtered_statsdata(appId, annual_statsDataId, "059", "消費支出")
surplus_df = get_filtered_statsdata(appId, annual_statsDataId, "234", "黒字")
# 世帯主の年齢、可処分所得、消費支出のデータを結合
on = ["時間軸（年次）", "世帯主の年齢階級"]
fie_df = age_df.merge(income_df, on=on).merge(expenditure_df, on=on)
fie_df.head()

e-Stat APIエラー (Status 0): 正常に終了しました。
e-Stat APIエラー (Status 0): 正常に終了しました。
e-Stat APIエラー (Status 0): 正常に終了しました。


,時間軸（年次）,世帯主の年齢階級,世帯主の年齢,可処分所得,消費支出
0,2024年,34歳以下,31.0,495530.0,271148.0
1,2024年,35～39歳,37.1,533352.0,288513.0
2,2024年,40～44歳,42.0,564760.0,311400.0
3,2024年,45～49歳,47.1,576439.0,348895.0
4,2024年,50～54歳,51.9,561118.0,364645.0


In [31]:
id_var = "世帯主の年齢"
value_vars = ["可処分所得", "消費支出"]
long_df = pd.melt(
    fie_df[[id_var] + value_vars],
    id_vars=id_var,
    value_vars=value_vars,
    var_name="収支",
    value_name="金額",
)

In [32]:
# 可処分所得と消費支出の折れ線グラフ
fie_fig = px.line(long_df, x="世帯主の年齢", y="金額", color="収支", markers=True)
# 黒字を棒グラフで追加
fie_fig.add_trace(
    go.Bar(x=age_df["世帯主の年齢"], y=surplus_df["黒字"], name="黒字", width=2.0)
)
# レイアウトに追加
fie_fig.update_layout(width=600, height=400, font={"size": 18},
    yaxis={"tickformat": ",.0f", "ticksuffix": "円", "title": None},
    xaxis={"ticksuffix": "歳"},
)
fie_fig.show()

#### 支出ツリーマップ

In [33]:
oneperson_statsDataId = "0003000795"
oneperson_meta = get_metainfo(appId, oneperson_statsDataId)
oneperson_metadata = oneperson_meta["GET_META_INFO"]["METADATA_INF"]
oneperson_total_num = oneperson_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
oneperson_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


388182

In [34]:
oneperson_params = {
    "cdCat02": "22",  # 世帯区分を（単身）勤労者世帯で絞る
    "cdCat03": "A00",  # 世帯主の年齢階級を平均で絞る
    "cdCat04": "0",  # 性別を男女平均で絞る
    "cdTimeFrom": "2024000000",  # 時間軸（四半期）を2024以降で絞る
    "cdTimeTo": "2025000000",  # 時間軸（四半期）を2025以前で絞る
}
oneperson_data = get_statsdata(appId, oneperson_statsDataId, params=oneperson_params)
oneperson_value = colname_to_japanese(cleansing_statsdata(oneperson_data))
oneperson_df = oneperson_value.loc[
    oneperson_value["単位"] == "円", ["用途分類コード", "時間軸（四半期）コード", "値"]
]

e-Stat APIエラー (Status 0): 正常に終了しました。


In [35]:
household_levels = create_hierarchy_dataframe(annual_meta, cat_key=1)
household_levels.iloc[35:45, 5:]

,level6,level7,用途分類階層1,用途分類階層2,用途分類階層3,用途分類階層4,用途分類階層5,用途分類階層6,用途分類階層7
35,049,049,018_受取,044_実収入以外の受取（繰入金を除く）,049_有価証券売却,049_有価証券売却,049_有価証券売却,049_有価証券売却,049_有価証券売却
36,050,050,018_受取,044_実収入以外の受取（繰入金を除く）,050_土地家屋借入金,050_土地家屋借入金,050_土地家屋借入金,050_土地家屋借入金,050_土地家屋借入金
37,051,051,018_受取,044_実収入以外の受取（繰入金を除く）,051_他の借入金,051_他の借入金,051_他の借入金,051_他の借入金,051_他の借入金
38,269,269,018_受取,044_実収入以外の受取（繰入金を除く）,269_クレジット購入借入金,269_クレジット購入借入金,269_クレジット購入借入金,269_クレジット購入借入金,269_クレジット購入借入金
39,052,052,018_受取,044_実収入以外の受取（繰入金を除く）,052_分割払購入借入金,052_分割払購入借入金,052_分割払購入借入金,052_分割払購入借入金,052_分割払購入借入金
40,053,053,018_受取,044_実収入以外の受取（繰入金を除く）,053_一括払購入借入金,053_一括払購入借入金,053_一括払購入借入金,053_一括払購入借入金,053_一括払購入借入金
41,054,054,018_受取,044_実収入以外の受取（繰入金を除く）,054_財産売却,054_財産売却,054_財産売却,054_財産売却,054_財産売却
42,055,055,018_受取,044_実収入以外の受取（繰入金を除く）,055_実収入以外の受取のその他,055_実収入以外の受取のその他,055_実収入以外の受取のその他,055_実収入以外の受取のその他,055_実収入以外の受取のその他
43,056,056,018_受取,056_繰入金,056_繰入金,056_繰入金,056_繰入金,056_繰入金,056_繰入金
44,062,062,057_支払,058_実支出,059_消費支出,060_食料,061_穀類,062_米,062_米


In [36]:
merged_df = oneperson_df.merge(
    household_levels, left_on="用途分類コード", right_on="level6", how="inner"
)

In [37]:
oneperson_value.loc[
    oneperson_value["用途分類"].str.contains("再掲"),
    ["用途分類コード", "用途分類", "用途分類階層レベル"],
].drop_duplicates()

,用途分類コード,用途分類,用途分類階層レベル
600,183,（再掲）教養娯楽関係費,4
604,186,（再掲）情報通信関係費,4
608,266,（再掲）消費支出（除く住居等）,4
612,187,（再掲）財・サービス計,4


In [38]:
lv_cols = [f"用途分類階層{i}" for i in range(2, 8) if i != 3]
# 受取018、可処分所得233、黒字234、繰越金216を除外
tree_cond = ~merged_df["level1"].isin(["018", "233", "234", "216"])
# 実支出"058"に絞る
tree_cond &= merged_df["level2"] == "058"
tree_cond &= ~merged_df["用途分類階層4"].str.contains("再掲")
tree_df = merged_df.loc[tree_cond, lv_cols + ["値"]]
tree_crosstab = (tree_df
    .groupby(lv_cols)
    .agg({"値": "sum"})
    .reset_index()
)
tree_crosstab.head()

,用途分類階層2,用途分類階層4,用途分類階層5,用途分類階層6,用途分類階層7,値
0,058_実支出,060_食料,061_穀類,062_米,062_米,1932.0
1,058_実支出,060_食料,061_穀類,063_パン,063_パン,4901.0
2,058_実支出,060_食料,061_穀類,064_麺類,064_麺類,2895.0
3,058_実支出,060_食料,061_穀類,065_他の穀類,065_他の穀類,658.0
4,058_実支出,060_食料,066_魚介類,067_生鮮魚介,067_生鮮魚介,2777.0


In [40]:
tree_fig = px.treemap(tree_crosstab, path=lv_cols, values="値")
tree_fig.data[0].textinfo = "label+value+percent root+percent parent"
tree_fig.show()

#### 家計資産・負債の可視化


In [41]:
balance_statsDataId = "0002210017"
balance_meta = get_metainfo(appId, balance_statsDataId)
balance_metadata = balance_meta["GET_META_INFO"]["METADATA_INF"]
balance_total_num = balance_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
balance_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


57984

In [42]:
balance_data = get_statsdata(appId, balance_statsDataId)
balance_value = colname_to_japanese(cleansing_statsdata(balance_data))
balance_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '貯蓄・負債コード', '世帯区分コード', '世帯主の年齢階級コード', '世帯の負債有無コード',
       '地域区分コード', '時間軸（四半期）コード', '単位', '値', '表章項目', '表章項目階層レベル', '貯蓄・負債',
       '貯蓄・負債階層レベル', '貯蓄・負債単位', '貯蓄・負債親コード', '世帯区分', '世帯区分階層レベル', '世帯主の年齢階級',
       '世帯主の年齢階級階層レベル', '世帯の負債有無', '世帯の負債有無階層レベル', '地域区分', '地域区分階層レベル',
       '時間軸（四半期）', '時間軸（四半期）階層レベル'],
      dtype='object')

In [43]:
balance_value = balance_value.assign(**{
    "年": balance_value["時間軸（四半期）コード"].astype(int) // 1_000_000
})
balance_cond = balance_value["時間軸（四半期）"] == "2024年1～3月期"
balance_cond &= balance_value["単位"] == "万円"
balance_cond &= (
    balance_value["世帯区分"] == "二人以上の世帯のうち勤労者世帯（2000年～）"
)
balance_cond &= ~balance_value["世帯主の年齢階級"].isin(["59歳以下", "60歳以上"])
balance_df = balance_value.loc[
    balance_cond, ["世帯主の年齢階級", "貯蓄・負債コード", "値"]
]

In [44]:
balance_levels = create_hierarchy_dataframe(balance_meta, cat_key=1, level_to=3)

In [45]:
merged_balance = balance_df.merge(
    balance_levels, left_on="貯蓄・負債コード", right_on="level3", how="inner"
)
balance_cols = ["世帯主の年齢階級", "貯蓄・負債階層1", "貯蓄・負債階層3"]
balance_crosstab = (
    merged_balance[balance_cols + ["値"]].groupby(balance_cols).sum().reset_index()
)
balance_crosstab.head()

,世帯主の年齢階級,貯蓄・負債階層1,貯蓄・負債階層3,値
0,29歳以下,011_年間収入,011_年間収入,705.0
1,29歳以下,012_貯蓄,014_通貨性預貯金,438.0
2,29歳以下,012_貯蓄,017_定期性預貯金,173.0
3,29歳以下,012_貯蓄,020_生命保険など,109.0
4,29歳以下,012_貯蓄,021_有価証券,68.0


In [46]:
# 後でグラフのX軸として使用する
age_cats = np.sort(merged_balance["世帯主の年齢階級"].unique())
# グラフで貯蓄の各カテゴリごとにデータを分類して表示するために使用
asset_cats = merged_balance.loc[
    merged_balance["貯蓄・負債階層1"] == "012_貯蓄", "貯蓄・負債階層3"
].unique()
# グラフで負債の各カテゴリごとにデータを分類して表示するために使用
liability_cats = merged_balance.loc[
    merged_balance["貯蓄・負債階層1"] == "028_負債", "貯蓄・負債階層3"
].unique()
# 貯蓄カテゴリのデータを表示する棒グラフで使用。各カテゴリが異なる青系の色で表示
asset_colors = plotly.colors.sequential.Blues[2 : len(asset_cats) + 2]
# 負債カテゴリのデータを表示する棒グラフで使用。各カテゴリが異なる赤系の色で表示
liability_colors = plotly.colors.sequential.OrRd[2 : len(liability_cats) + 2]
print("年齢リスト: ", age_cats, "\n資産リスト: ", asset_cats, "\n負債リスト: ", liability_cats)

年齢リスト:  ['29歳以下' '30～39歳' '40～49歳' '50～59歳' '60～69歳' '70歳以上'] 
資産リスト:  ['014_通貨性預貯金' '017_定期性預貯金' '020_生命保険など' '021_有価証券' '025_金融機関外' '026_年金型貯蓄'
 '027_外貨預金・外債'] 
負債リスト:  ['030_公的（住宅・土地の負債）' '031_民間（住宅・土地の負債）' '032_その他（住宅・土地の負債）'
 '034_公的（住宅・土地以外負債）' '035_民間（住宅・土地以外負債）' '036_その他（住宅・土地以外負債）' '037_月賦・年賦']


In [47]:
balance_fig = go.Figure()
lv = "貯蓄・負債階層3"
# 最初の資産カテゴリに対応する値を取得
asset_val = balance_crosstab.loc[balance_crosstab[lv] == asset_cats[0], "値"]
# 最初の資産カテゴリに対応する棒グラフを追加
balance_fig.add_trace(go.Bar(
    x=age_cats,
    y=asset_val.values,
    name=asset_cats[0],
    marker_color=asset_colors[0],
    offsetgroup=1,  # オフセットグループを1に設定して負債グループと区別
))  
# 最初の資産カテゴリの値を基準としてコピーし、積み上げの基準とする
asset_stack = asset_val.copy().values
# 最初の資産カテゴリを除く各資産カテゴリについてループ
for i in range(len(asset_cats) - 1):
    stack_values = balance_crosstab.loc[
        balance_crosstab[lv] == asset_cats[i + 1], "値"
    ].values
    # 次(i+1)の資産カテゴリに対応する棒グラフを追加
    balance_fig.add_trace(go.Bar(
        x=age_cats,
        y=stack_values,
        name=asset_cats[i + 1],
        marker_color=asset_colors[i + 1],
        base=asset_stack,  # 前の資産カテゴリの値を基準にして積み上げ
        offsetgroup=1,  # オフセットグループを1に設定
    ))
    # 積上値の更新。前のカテゴリの値にこのカテゴリの値を加算して、次の積上の基準にする
    asset_stack += stack_values
# 最初の負債カテゴリに対応する値を取得
liability_val = balance_crosstab.loc[balance_crosstab[lv] == liability_cats[0], "値"]
# 最初の負債カテゴリに対応する棒グラフを追加
balance_fig.add_trace(go.Bar(
    x=age_cats,
    y=liability_val,
    name=liability_cats[0],
    marker_color=liability_colors[0],
    offsetgroup=2,  # オフセットグループを1に設定して資産グループと区別
))
# 最初の負債カテゴリの値を基準としてコピーし、積み上げの基準とする
liability_stack = liability_val.copy().values
# 最初の負債カテゴリを除く各負債カテゴリについてループ
for i in range(len(liability_cats) - 1):
    stack_values = balance_crosstab.loc[
        balance_crosstab[lv] == liability_cats[i + 1], "値"
    ].values
    # 次(i+1)の負債カテゴリに対応する棒グラフを追加
    balance_fig.add_trace(go.Bar(
        x=age_cats,
        y=stack_values,
        name=liability_cats[i + 1],
        marker_color=liability_colors[i + 1],
        base=liability_stack,
        offsetgroup=2,
    ))
    # 積上値の更新。前のカテゴリの値にこのカテゴリの値を加算して、次の積上の基準にする
    liability_stack += stack_values
balance_fig.update_layout(
    go.Layout(width=700, height=500, font={"size": 14},
        barmode="group",
        yaxis={"tickformat": ".0f", "ticksuffix": "万円"},
    )
)
balance_fig.show()

### 全国家計構造調査

- [全国家計構造調査](https://www.stat.go.jp/data/zenkokukakei/2024/index.html)


In [48]:
ficw_statsDataId = "0003424621"
ficw_meta = get_metainfo(appId, ficw_statsDataId)
ficw_metadata = ficw_meta["GET_META_INFO"]["METADATA_INF"]
ficw_total_num = ficw_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
ficw_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


473472

In [49]:
cat4 = pd.DataFrame(ficw_metadata["CLASS_INF"]["CLASS_OBJ"][4]["CLASS"])
cat4.iloc[380:390, :]

,@code,@name,@level,@parentCode
380,320203,他の保険金,4,3202
381,3203,有価証券売却,3,32
382,3204,土地家屋借入金,3,32
383,3205,他の借入金,3,32
384,3206,クレジット購入借入金,3,32
385,3207,財産売却,3,32
386,3208,実収入以外の受取のその他,3,32
387,33,繰入金,2,3
388,4,可処分所得,1,NaN
389,5,（特掲：用途分類）交際費,1,NaN


#### 年齢ごと収支


In [50]:
# 世帯主の年齢階級32区分ごとの家計項目データ取得のための関数を用意
def get_statsdata_by_cdCat04(
        appId: str, statsDataId: str, cdCat04: str, name: str
    ) -> pd.DataFrame:
    params = {
        "cdCat01": "1",  # 世帯の種類３区分を二人以上の世帯で絞る
        "cdCat02": "1",  # 世帯区分４区分を勤労者世帯で絞る
        "cdCat03": "0",  # 世帯主の性別３区分を平均で絞る
        "cdCat04": cdCat04,
    }
    data = get_statsdata(appId, statsDataId, params=params)
    value = colname_to_japanese(cleansing_statsdata(data))
    cond = value["世帯主の年齢階級32区分"].str.contains("５歳階級")
    df = value.loc[cond, ["世帯主の年齢階級32区分", "値"]]
    return df.rename(columns={"値": name})
# 可処分所得データの取得
income_df = get_statsdata_by_cdCat04(appId, ficw_statsDataId, "4", "可処分所得")
# 消費支出データの取得
expenditure_df = get_statsdata_by_cdCat04(appId, ficw_statsDataId, "2101", "消費支出")
# 両データを「世帯主の年齢階級32区分」で結合
ficw_df = pd.merge(income_df, expenditure_df, on="世帯主の年齢階級32区分")
# データを整然データ形式に変換
ficw_crosstab = (ficw_df
    .set_index("世帯主の年齢階級32区分")
    .stack()
    .reset_index()
    .rename(columns={"level_1": "収支", 0: "金額"})
)
ficw_crosstab.head()

e-Stat APIエラー (Status 0): 正常に終了しました。
e-Stat APIエラー (Status 0): 正常に終了しました。


,世帯主の年齢階級32区分,収支,金額
0,30歳未満（５歳階級）,可処分所得,365708.0
1,30歳未満（５歳階級）,消費支出,227574.0
2,30～34（５歳階級）,可処分所得,403435.0
3,30～34（５歳階級）,消費支出,253233.0
4,35～39（５歳階級）,可処分所得,429951.0


In [51]:
ficw_fig = px.line(ficw_crosstab, x="世帯主の年齢階級32区分", y="金額", color="収支", markers=True)
ficw_fig.update_layout(width=900, height=400, font={"size": 18},
    xaxis={"title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "円"})
ficw_fig.show()

## 生活に関する統計データ

###  国民生活基礎調査

- [国民生活基礎調査](https://www.mhlw.go.jp/toukei/list/20-21.html)


In [52]:
living_statsDataId = "0002042803"
living_meta = get_metainfo(appId, living_statsDataId)
living_metadata = living_meta["GET_META_INFO"]["METADATA_INF"]
living_total_num = living_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
living_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


240

In [53]:
living_data = get_statsdata(appId, living_statsDataId)
living_value = colname_to_japanese(cleansing_statsdata(living_data))
living_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '世帯類型_103コード', '所得金額階級_2015コード', '調査年度コード', '単位', '値',
       '表章項目', '表章項目階層レベル', '表章項目単位', '世帯類型_103', '世帯類型_103階層レベル',
       '世帯類型_103親コード', '所得金額階級_2015', '所得金額階級_2015階層レベル', '所得金額階級_2015親コード',
       '調査年度', '調査年度階層レベル'],
      dtype='object')

In [54]:
living_cols = ["世帯類型_103", "所得金額階級_2015コード", "所得金額階級_2015", "値"]
living_cond = living_value["単位"] != "万円"
living_cond &= living_value["世帯類型_103"].isin(["総数", "高齢者世帯以外の世帯"])
living_cond &= living_value["所得金額階級_2015"] != "総数"
living_df = living_value.loc[living_cond, living_cols]
living_df.head()

,世帯類型_103,所得金額階級_2015コード,所得金額階級_2015,値
1,総数,120,５０万円未満,1.2
2,総数,130,５０～１００万円未満,5.5
3,総数,140,１００～１５０万円未満,6.4
4,総数,150,１５０～２００万円未満,6.6
5,総数,170,２００～２５０万円未満,7.7


In [55]:
pivot_df = living_df.pivot(
    columns="世帯類型_103",
    index=["所得金額階級_2015コード", "所得金額階級_2015"],
    values="値",
)
# 所得金額階級の降順でソートし、各列について累積和を計算
cumsum_df = pivot_df.sort_index(level=0, ascending=False).cumsum()
# cumsum_dfをロングテーブル形式にし、パーセントのデータが入る列名を "パーセント" に変更
icdf_df = cumsum_df.stack().reset_index().rename(columns={0: "パーセント"})

In [56]:
# 所得金額階級分布
dist_fig = px.bar(
    living_df, x="所得金額階級_2015", y="値", color="世帯類型_103", barmode="group"
)
dist_fig.update_layout(
    yaxis={"tickformat": ".0f", "ticksuffix": "%", "title": None},
    legend_title_text="世帯類型"
)

In [57]:
# 所得金額階級の逆累積分布
icdf_fig = px.bar(
    icdf_df, x="所得金額階級_2015", y="パーセント", color="世帯類型_103", barmode="group"
)
icdf_fig.update_layout(
    yaxis={"tickformat": ".0f", "ticksuffix": "%", "title": None},
    legend_title_text="世帯類型"
)

In [58]:
# サブプロットを作成
subplot_fig = make_subplots(rows=1, cols=2, subplot_titles=("所得金額分布", "逆累積分布"))
# 各グラフの trace を対応セルに追加
for trace in dist_fig.data:
    subplot_fig.add_trace(trace, row=1, col=1)
for trace in icdf_fig.data:
    trace.showlegend = False
    subplot_fig.add_trace(trace, row=1, col=2)
# 全体レイアウト調整
subplot_fig.update_layout(width=1400, height=600, font={"size": 14}, 
    title_text="所得金額階級", legend_title_text="世帯類型")
subplot_fig.update_xaxes(title_text=None, row=1, col=1)
subplot_fig.update_yaxes(tickformat=".0f", ticksuffix="%", title_text=None, row=1, col=1)
subplot_fig.update_xaxes(title_text=None, row=1, col=2)
subplot_fig.update_yaxes(tickformat=".0f", ticksuffix="%", title_text=None, row=1, col=2)
subplot_fig.show()

### 社会生活基本調査と生活時間

- [社会生活基本調査](https://www.stat.go.jp/data/shakai/2021/index.htm)


In [59]:
timeuse_statsDataId = "0003462382"
timeuse_meta = get_metainfo(appId, timeuse_statsDataId)
timeuse_metadata = timeuse_meta["GET_META_INFO"]["METADATA_INF"]
timeuse_total_num = timeuse_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
timeuse_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


622080

In [60]:
cat06_meta = pd.DataFrame(
    timeuse_metadata["CLASS_INF"]["CLASS_OBJ"][6]["CLASS"]
)
cat06_meta.head()

,@code,@name,@level
0,0,0_総数,1
1,1,1_有償労働,1
2,11,11_主な仕事関連,2
3,111,111_主な仕事,3
4,112,112_主な仕事中の移動,3


#### 生活時間円グラフ

In [61]:
male_params = {
    "cdCat01": "1",  # 曜日を"1_週全体"に絞る
    "cdCat02": "1",  # 男女を"1_男"に絞る
    "cdCat03": "0",  # ふだんの就業状態を"0_総数"に絞る
    "cdCat04": "0",  # ふだんの健康状態を"0_総数"に絞る
    "cdCat05": "0",  # 年齢を"0_総数"に絞る
    "lvCat06": "2",  # 行動の種類をレベル2に絞る
}
male_data = get_statsdata(appId, timeuse_statsDataId, params=male_params)
male_value = colname_to_japanese(cleansing_statsdata(male_data))
male_df = male_value.loc[:, ["行動の種類", "値"]]
male_df.head()

e-Stat APIエラー (Status 0): 正常に終了しました。


,行動の種類,値
0,11_主な仕事関連,275.0
1,12_副業関連,1.0
2,13_通勤,35.0
3,14_その他の仕事関連,4.0
4,21_家事,46.0


In [62]:
male_hours = (male_df["値"] / 60).astype(str).str[:3] + "時間"
male_minutes = male_df["値"].astype(int).astype(str) + "分"
male_timeuse = male_minutes.mask(male_df["値"] / 60 >= 1, male_hours)
male_df = male_df.assign(**{
    "ラベル": male_df["行動の種類"].str[3:] + ":" + male_timeuse
})

In [63]:
female_params = {
    "cdCat01": "1",
    "cdCat02": "2",  # 男女を"2_女"に絞る
    "cdCat03": "0",
    "cdCat04": "0",
    "cdCat05": "0",
    "lvCat06": "2",
}
female_data = get_statsdata(appId, timeuse_statsDataId, params=female_params)
female_value = colname_to_japanese(cleansing_statsdata(female_data))
female_df = female_value.loc[:, ["行動の種類", "値"]]
female_hours = (female_df["値"] / 60).astype(str).str[:3] + "時間"
female_minutes = female_df["値"].astype(int).astype(str) + "分"
female_timeuse = female_minutes.mask(female_df["値"] / 60 >= 1, female_hours)
female_df = female_df.assign(**{
    "ラベル": female_df["行動の種類"].str[3:] + ":" + female_timeuse
})

e-Stat APIエラー (Status 0): 正常に終了しました。


In [64]:
pie_colors = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel
male_trace = go.Pie(
    labels=male_df["ラベル"],
    values=male_df["値"],
    marker={"colors": pie_colors},  # カラーパレットをグラフの色として設定
    textinfo="label",  # 各セグメントにラベルを表示
    sort=False,  # データのソートを行わない
    rotation=0,  # グラフの回転を0度に設定
)
female_trace = go.Pie(
    labels=female_df["ラベル"],
    values=female_df["値"],
    marker={"colors": pie_colors},
    textinfo="label",
    sort=False,
    rotation=0,
)
# 1行2列のサブプロットを作成し、各サブプロットに円グラフタイプを指定
pie_fig = make_subplots(rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],  # 各サブプロットに円グラフタイプを指定
    subplot_titles=("男性", "女性"),  # 各サブプロットのタイトルを設定
    horizontal_spacing=0.3,  # サブプロット間の水平方向の間隔を設定
)  
# 作成した男性データの円グラフを1列目に、女性データを2列目に配置
pie_fig.add_trace(male_trace, row=1, col=1)
pie_fig.add_trace(female_trace, row=1, col=2)
# 全ての円グラフの描画方向を時計回りに設定
pie_fig.update_traces(direction="clockwise", selector={"type": "pie"})
pie_fig.update_layout(width=1400, height=650, font={"size": 14},
    showlegend=False
)
pie_fig.show()

#### 生活時間サンバーストグラフ

In [65]:
# "0_総数"、"R1_(再掲)無償労働(国際比較)"を除外
cat06_cond = ~cat06_meta["@code"].isin(["0", "R1"])
# 階層1について
lv_cond = cat06_meta["@code"].str.len() == 1  # コードの長さが1のものを選択
timeuse_levels = cat06_meta.loc[cat06_cond & lv_cond, ["@code"]]
timeuse_levels = timeuse_levels.set_axis(["level1"], axis=1)  # 抽出した列の列名を "level1" に変更
# 階層2,3についてループ処理
for lv in range(2, 4):
    lv_cond = cat06_meta["@code"].str.len() == lv  # コードの長さがlvの条件を設定
    child_code = cat06_meta.loc[cat06_cond & lv_cond, ["@code"]]
    child_code.columns = ["level" + str(lv)]
    # 抽出した列の列名をその階層のlevel名に変更
    child_code["level" + str(lv - 1)] = child_code["level" + str(lv)].str[: (lv - 1)]
    timeuse_levels = timeuse_levels.merge(
        child_code, on="level" + str(lv - 1), how="left"
    )  # 既存のlevelsデータフレームに新しく作成したchild_codeをマージ
# レベル1から3までの各レベルでメタデータと結合し、行動の種類を列名に設定
for lv in range(1, 4):
    timeuse_levels = timeuse_levels.merge(
        cat06_meta[["@code", "@name"]],
        left_on="level" + str(lv),
        right_on="@code",
        how="left",
    )
    timeuse_levels = timeuse_levels.drop("@code", axis=1).rename(
        columns={"@name": "行動の種類" + str(lv)}
    )
# Nullを前方から補完
timeuse_levels = timeuse_levels.ffill(axis=1)
timeuse_levels.head()

,level1,level2,level3,行動の種類1,行動の種類2,行動の種類3
0,1,11,111,1_有償労働,11_主な仕事関連,111_主な仕事
1,1,11,112,1_有償労働,11_主な仕事関連,112_主な仕事中の移動
2,1,12,121,1_有償労働,12_副業関連,121_副業
3,1,12,122,1_有償労働,12_副業関連,122_副業中の移動
4,1,13,131,1_有償労働,13_通勤,131_通勤


In [66]:
timeuse_params = {
    "cdCat01": "1",  # 曜日を1_週全体に絞る
    "cdCat02": "1",  # 男女を1_男に絞る
    "cdCat03": "0",  # ふだんの就業状態を'0_総数'に絞る
    "cdCat04": "0",  # ふだんの健康状態を'0_総数'に絞る
    "cdCat05": "0",  # 年齢を'0_総数'に絞る
}
timeuse_data = get_statsdata(appId, timeuse_statsDataId, params=timeuse_params)
timeuse_value = colname_to_japanese(cleansing_statsdata(timeuse_data))

e-Stat APIエラー (Status 0): 正常に終了しました。


In [67]:
act_list = [f"行動の種類{i}" for i in range(1, 4)]
timeuse_df = timeuse_value.loc[
    cat06_cond, ["行動の種類コード", "行動の種類", "行動の種類階層レベル", "値"]
]
lvmerged_timeuse = timeuse_df.merge(
    timeuse_levels, left_on="行動の種類", right_on="行動の種類3", how="left"
)
sunburst_df = lvmerged_timeuse.loc[lvmerged_timeuse["行動の種類3"].notnull(), act_list + ["値"]]
sunburst_df.head()

,行動の種類1,行動の種類2,行動の種類3,値
2,1_有償労働,11_主な仕事関連,111_主な仕事,269.0
3,1_有償労働,11_主な仕事関連,112_主な仕事中の移動,6.0
5,1_有償労働,12_副業関連,121_副業,1.0
6,1_有償労働,12_副業関連,122_副業中の移動,0.0
8,1_有償労働,13_通勤,131_通勤,35.0


In [68]:
sunburst_fig = px.sunburst(sunburst_df, path=act_list, values="値")
# サンバースト図の各セグメントに、ラベルとそのセグメントが全体に占める割合を表示する設定を追加
sunburst_fig.update_traces(textinfo="label+percent root")
sunburst_fig.show()

#### 生活時間エリアチャート


In [69]:
area_params = {
    "cdCat01": "1",  # 曜日を"1_週全体"に絞る
    "cdCat02": "1",  # 男女を"1_男"に絞る
    "cdCat03": "0",  # ふだんの就業状態を"0_総数"に絞る
    "cdCat04": "0",  # ふだんの健康状態を"0_総数"に絞る
    "lvCat06": "2",  # 行動の種類をレベル2に絞る
}
area_data = get_statsdata(appId, timeuse_statsDataId, params=area_params)
area_value = colname_to_japanese(cleansing_statsdata(area_data))
# 年齢と行動の種類の重複して集計してしまう項目を除外
area_cond = ~area_value["行動の種類コード"].isin(
    ["0", "R1"]
)  # "0_総数"、"R1_(再掲)無償労働(国際比較)"を除外
area_cond &= ~area_value["年齢"].isin(
    ["0_総数", "R1_(再掲)35歳以上", "R2_(再掲)65歳以上"]
)
area_df = area_value.loc[area_cond, ["年齢", "行動の種類", "値"]]
# 時間を分に変換
area_df = area_df.assign(**{"時間": area_df["値"] / 60})
area_df.head()

e-Stat APIエラー (Status 0): 正常に終了しました。


,年齢,行動の種類,値,時間
22,1_15～24歳,11_主な仕事関連,129.0,2.150000
23,1_15～24歳,12_副業関連,0.0,0.000000
24,1_15～24歳,13_通勤,18.0,0.300000
25,1_15～24歳,14_その他の仕事関連,2.0,0.033333
26,1_15～24歳,21_家事,15.0,0.250000


In [70]:
area_colors = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel
area_fig = px.area(
    area_df, x="年齢", y="時間", color="行動の種類", color_discrete_sequence=area_colors
)
area_fig.update_layout(width=1000, height=650, font={"size": 14})
area_fig.show()

## 労働に関する統計データ

- [労働力調査](https://www.stat.go.jp/data/roudou/)
- [就業構造基本調査](https://www.stat.go.jp/data/shugyou/2022/index.html)
- [毎月勤労統計調査](https://www.mhlw.go.jp/toukei/list/30-1.html)
- [賃金構造基本統計調査](https://www.mhlw.go.jp/toukei/list/chinginkouzou.html)
- [労働統計局](https://www.bls.gov/)


### 労働力調査と失業率

- [労働力調査](https://www.stat.go.jp/data/roudou/index.html)
- [労働力調査の解説](https://www.stat.go.jp/data/roudou/pdf/hndbk.pdf)
- [就業構造基本調査](https://www.stat.go.jp/data/shugyou/2022/gaiyou.html)


In [71]:
labour_force_statsDataId = "0003005865" 
labour_meta = get_metainfo(appId, labour_force_statsDataId)
labour_metadata = labour_meta["GET_META_INFO"]["METADATA_INF"]
labour_total_num = labour_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
labour_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


2475

In [72]:
labour_data = get_statsdata(appId, labour_force_statsDataId)
labour_value = colname_to_japanese(cleansing_statsdata(labour_data))
labour_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '産業コード', '就業状態コード', '性別コード', '地域コード', '時間軸（月次）コード', '単位',
       '値', '表章項目', '表章項目階層レベル', '表章項目単位', '産業', '産業階層レベル', '就業状態',
       '就業状態階層レベル', '性別', '性別階層レベル', '地域', '地域階層レベル', '時間軸（月次）',
       '時間軸（月次）階層レベル'],
      dtype='object')

In [73]:
labour_value = labour_value.assign(**{
    "年": labour_value["時間軸（月次）コード"].str[:4],
    "月": labour_value["時間軸（月次）コード"].str[-2:],
    "年月": lambda df: pd.to_datetime(df["年"] + "-" + df["月"])
})
labour_cond = labour_value["就業状態"] == "完全失業者"
labour_cond &= labour_value["性別"] == "総数"
labour_df = labour_value.loc[labour_cond, ["年月", "値"]]

### 米国雇用統計

- [労働統計局](https://www.bls.gov/)
- [BLS Public Data API](https://www.bls.gov/developers/)
- [registration page](https://www.bls.gov/developers/api_faqs.htm?#register1)
- [Help & Tutorials](https://www.bls.gov/help/hlpforma.htm)
- [Accessing the Public Data API with Python](https://www.bls.gov/developers/api_python.htm)



In [74]:
# BLS APIを使用してデータを取得する関数を定義
def get_bls_data(
        bls_api_key: str, series_id: str, start_year: int, end_year: int
    ) -> pd.DataFrame:
    bls_data_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    bls_headers = {"Content-Type": "application/json"}
    series_ids = [series_id]
    bls_payload = {
        "registrationkey": bls_api_key,
        "seriesid": series_ids,
        "startyear": start_year,
        "endyear": end_year,
        "catalog": True,  # データカタログを含める
    }
    bls_res = requests.post(bls_data_url, data=json.dumps(bls_payload), headers=bls_headers)
    bls_json = bls_res.json()
    return bls_json

bls_json = get_bls_data(bls_api_key, "LNS14000000", 2000, 2010)

In [75]:
lns_data1 = pd.DataFrame(bls_json["Results"]["series"][0]["data"])
bls_json2 = get_bls_data(bls_api_key, "LNS14000000", 2011, 2024)
lns_data2 = pd.DataFrame(bls_json2["Results"]["series"][0]["data"])
lns_data = pd.concat([lns_data1, lns_data2], ignore_index=True)
lns_data.head()

,year,period,periodName,value,footnotes
0,2010,M12,December,9.3,[{}]
1,2010,M11,November,9.8,[{}]
2,2010,M10,October,9.4,[{}]
3,2010,M09,September,9.5,[{}]
4,2010,M08,August,9.5,[{}]


In [76]:
lns_df = lns_data.assign(
    month=lns_data["period"].str.extract(r"M(\d+)").astype(int)
).assign(
    date=lambda df: pd.to_datetime(
        df["year"].astype(str) + "-" + df["month"].astype(str) + "-01")
).sort_values(by="date")
unrate = (lns_df[["date", "value"]].copy()
    .rename(columns={"date": "年月", "value": "失業率"})
    .assign(**{"国": "米国"})
)
unrate.head()

,年月,失業率,国
131,2000-01-01,4.0,米国
130,2000-02-01,4.1,米国
129,2000-03-01,4.0,米国
128,2000-04-01,3.8,米国
127,2000-05-01,4.0,米国


In [77]:
unrate_jp = (labour_df
    .rename(columns={"値": "失業率"})
    .assign(**{"国": "日本"})
)
unrate_df = pd.concat([unrate_jp, unrate])

In [78]:
labour_fig = px.line(unrate_df, x="年月", y="失業率", color="国")
labour_fig.update_layout(width=900, height=500, font={"size": 18},
    xaxis={"ticksuffix": "年"},
    yaxis={"ticksuffix": "%"}
)
labour_fig.show()

### 毎月勤労統計と賃金

- [賃金構造基本調査](https://www.mhlw.go.jp/toukei/list/chinginkouzou.html)
- [毎月勤労統計調査](https://www.e-stat.go.jp/stat-search?page=1&toukei=00450071)
- [賃金構造基本統計調査と毎月勤労統計調査の相違](https://www.mhlw.go.jp/toukei/list/chinginkouzou_sankou_a.html)
- [労働力調査と毎月勤労統計調査の相違](https://www.stat.go.jp/data/roudou/qa-1.html)
- [毎月勤労統計調査全国調査で作成している指数等の解説](https://www.mhlw.go.jp/toukei/itiran/roudou/monthly/sisuu/sisuu_r04.html)
- [毎月勤労統計調査を巡る不適切な取扱いに係る 事実関係とその評価等に関する報告書](https://www.mhlw.go.jp/content/10108000/000472506.pdf)
- [毎月勤労統計調査を巡る不適切な取扱いに係る 事実関係とその評価等に関する追加報告書](https://www.soumu.go.jp/main_content/000605455.pdf)


In [79]:
monthly_labour = pd.read_csv(data_path / "hon-maikin-k-kicho.csv", encoding="shift_jis")
monthly_labour.columns

Index(['種別', '年', '月', '産業分類', '規模', '就業形態', '常用雇用', '現金給与総額', 'きまって支給する給与',
       '実質賃金（現金給与総額）', '総実労働時間', '所定外労働時間'],
      dtype='object')

In [80]:
monthly_labour["種別"].unique()

array(['季節調整済指数', '季節調整済指数伸び率（前月比）'], dtype=object)

In [81]:
wage_cond = monthly_labour["年"] >= 2019
wage_cond &= monthly_labour["年"] < 2025
wage_cond &= monthly_labour["種別"] == "季節調整済指数伸び率（前月比）"
wage_cond &= monthly_labour["産業分類"].str.startswith("TL")
wage_cond &= monthly_labour["規模"] == "T"
wage_cond &= monthly_labour["就業形態"] == 0
wage_ym = monthly_labour.loc[wage_cond, ["年", "月", "現金給与総額"]].assign(**{
    "年月": lambda df: pd.to_datetime(df["年"].astype(str) + "-" + df["月"].astype(str))
})
wage_ym.tail()

,年,月,現金給与総額,年月
8824,2024,8,-0.2,2024-08-01
8837,2024,9,0.0,2024-09-01
8850,2024,10,0.4,2024-10-01
8863,2024,11,0.7,2024-11-01
8876,2024,12,0.6,2024-12-01


In [82]:
wage = wage_ym[["年月", "現金給与総額"]].rename(columns={"現金給与総額": "賃金上昇率"})
unemployment_rate = labour_df.rename(columns={"値": "失業率"})
wp_df = wage.merge(unemployment_rate, on="年月", how="inner")

In [83]:
y = wage["年月"].min().year
m = wage["年月"].min().month
wpc_fig = px.scatter(wp_df.reset_index(), x="失業率", y="賃金上昇率", color="index")
wpc_fig.update_layout(
    xaxis={"ticksuffix": "%"},
    yaxis={"ticksuffix": "%"},
    coloraxis={"colorbar": {"title": {"text": f"経過月数({y}/{m}基準)"}}},
)
wpc_fig.show()